# Предсказание стоимости жилья

Цель проекта - обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Для оценки качества модели требуется использовать метрики RMSE, MAE и R2.

## Описание данных

Описание данных в файле `housing.csv`:
* `longitude` - широта;
* `latitude` - долгота;
* `housing_median_age` — медианный возраст жителей жилого массива;
* `total_rooms` — общее количество комнат в домах жилого массива;
* `total_bedrooms` — общее количество спален в домах жилого массива;
* `population` — количество человек, которые проживают в жилом массиве;
* `households` — количество домовладений в жилом массиве;
* `median_income` — медианный доход жителей жилого массива;
* `median_house_value` — медианная стоимость дома в жилом массиве, наш таргет;
* `ocean_proximity` — близость к океану.

## Ознакомление с данными 

### Импорты

In [35]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import col, mean

from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml import Pipeline

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

### Инициализация Spark-сессии

In [36]:
spark = SparkSession.builder \
    .appName("SparkHousing") \
    .getOrCreate()

### Просмотр схемы

In [37]:
file_path = "/datasets/housing.csv"
housing_data = spark.read.csv(file_path, header=True, inferSchema=True)

housing_data.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [38]:
housing_data.head()

Row(longitude=-122.23, latitude=37.88, housing_median_age=41.0, total_rooms=880.0, total_bedrooms=129.0, population=322.0, households=126.0, median_income=8.3252, median_house_value=452600.0, ocean_proximity='NEAR BAY')

## Предобработка данных

### Исследование пропусков

Пропуски наблюдаем в столбце `total_bedrooms`. Заполним их значением-заглушкой `0`.

In [39]:
housing_data = housing_data.na.fill(0)

### Разделение на выборки

In [40]:
#разделение на трейн и тест
(training_data, test_data) = housing_data.randomSplit([0.8, 0.2], seed=42)

### OHE категориальных данных

Из схемы видим, что категориальным признаком является только `ocean_proximity`

In [41]:
housing_data.select('ocean_proximity').distinct().collect()

[Row(ocean_proximity='ISLAND'),
 Row(ocean_proximity='NEAR OCEAN'),
 Row(ocean_proximity='NEAR BAY'),
 Row(ocean_proximity='<1H OCEAN'),
 Row(ocean_proximity='INLAND')]

In [43]:
indexer = StringIndexer(inputCol='ocean_proximity', outputCol='ocean_proximity_index')

indexer_model = indexer.fit(training_data)

indexed_train = indexer_model.transform(training_data)
indexed_test = indexer_model.transform(test_data)

In [44]:
encoder = OneHotEncoder(inputCol='ocean_proximity_index', outputCol='ocean_proximity_ohe')

encoder_model = encoder.fit(indexed_train)

training_data = encoder_model.transform(indexed_train)
test_data = encoder_model.transform(indexed_test)

In [48]:
training_data.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------------+-------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_index|ocean_proximity_ohe|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+---------------------+-------------------+
|  -124.35|   40.54|              52.0|     1820.0|         300.0|     806.0|     270.0|       3.0147|           94600.0|     NEAR OCEAN|                  2.0|      (4,[2],[1.0])|
|   -124.3|    41.8|              19.0|     2672.0|         552.0|    1298.0|     478.0|       1.9797|           85800.0|     NEAR OCEAN|                  2.0|      (4,[2],[1.0])|
|  -124.27|   40.69|              36.0|     2349.0|         528.0|    1194.0|     465.0|       2.517

## Построение моделей

Заказчик требует построить две модели на двух разных наборах данных:
* используя все данные из файла;
* используя только числовые переменные, исключив категориальные.
Построим для каждого случая пайплайн.

In [49]:
#модель на всех данных
feature_columns_all = [
    "longitude", "latitude", "housing_median_age", "total_rooms",
    "total_bedrooms", "population", "households", "median_income",
    "ocean_proximity_ohe"
]

assembler_all = VectorAssembler(inputCols=feature_columns_all, outputCol="features_all")
lr_all = LinearRegression(labelCol="median_house_value", featuresCol="features_all")
pipeline_all = Pipeline(stages=[assembler_all, lr_all])
model_all = pipeline_all.fit(training_data)

23/12/06 11:03:08 WARN Instrumentation: [4f59d61f] regParam is zero, which might cause numerical instability and overfitting.


In [50]:
#Модель на числовых данных
feature_columns_numeric = [
    "longitude", "latitude", "housing_median_age", "total_rooms",
    "total_bedrooms", "population", "households", "median_income"
]

assembler_numeric = VectorAssembler(inputCols=feature_columns_numeric, outputCol="features_numeric")
lr_numeric = LinearRegression(labelCol="median_house_value", featuresCol="features_numeric")
pipeline_numeric = Pipeline(stages=[assembler_numeric, lr_numeric])
model_numeric = pipeline_numeric.fit(training_data)

23/12/06 11:03:11 WARN Instrumentation: [099e92ee] regParam is zero, which might cause numerical instability and overfitting.


Проверим работу моделей на трех требуемых метриках:

In [51]:
#Оценка моделей на тестовых данных
evaluator_rmse = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="mae")
evaluator_r2 = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="r2")

predictions_all = model_all.transform(test_data)
rmse_all = evaluator_rmse.evaluate(predictions_all)
print("RMSE (все данные):", rmse_all)

predictions_numeric = model_numeric.transform(test_data)
rmse_numeric = evaluator_rmse.evaluate(predictions_numeric)
print("RMSE (численные данные):", rmse_numeric)

RMSE (все данные): 70895.92875824224
RMSE (численные данные): 71934.43916846818


In [52]:
mae_all = evaluator_mae.evaluate(predictions_all)
mae_numeric = evaluator_mae.evaluate(predictions_numeric)

print("MAE (все данные):", mae_all)
print("MAE (численные данные):", mae_numeric)

MAE (все данные): 50949.266663002214
MAE (численные данные): 51936.17292777196


In [53]:
r2_all = evaluator_r2.evaluate(predictions_all)
r2_numeric = evaluator_r2.evaluate(predictions_numeric)

print("R2 (все данные):", r2_all)
print("R2 (численные данные):", r2_numeric)

R2 (все данные): 0.6367242913742457
R2 (численные данные): 0.6260035412108075


In [54]:
#все результаты получены
spark.stop()

# Выводы

Данные были предобработаны и изучены при помощи методов pySpark. Было построено две модели линейной регрессии на разных наборах данных - на всех одновременно и только на численных. Для построения и оценки моделей была использована библиотека MLlib. Были получены следующие результаты:
* RMSE (все данные): 70895.92875824224 / RMSE (численные данные): 71934.43916846818
  
* MAE (все данные): 50949.266663002214 / MAE (численные данные): 51936.17292777196

* R2 (все данные): 0.6367242913742457 / R2 (численные данные): 0.6260035412108075

Как видим: 
*  Модель, обученная на всех данных, имеет меньшее значение RMSE, что свидетельствует о лучшей предсказательной способности по сравнению с моделью, обученной только на числовых данных.
* Здесь также модель, обученная на всех данных, имеет меньшее значение MAE, что говорит о лучшей предсказательной способности.
* Значение R2 ближе к 1 указывает на лучшую объяснительную способность модели. В данном случае модель, обученная на всех данных, имеет немного лучшее значение R2.

Модель, обученная на всех данных (включая категориальные признаки), выглядит более предпочтительной с точки зрения RMSE, MAE и R2 по сравнению с моделью, обученной только на числовых данных.